In [1]:
# required Python imports
import pandas as pd
import numpy as np
from types import SimpleNamespace

import ftir_funct as f
np.set_printoptions(suppress=True)

module FTIR v.2024.4.11 imported


## Generate database

In [2]:
f.explore_Euler_space(step=7, lower_bounds=(0, 0, 0), upper_bounds=(89, 89, 179))

array([[  0,   0,   0],
       [  0,   0,   7],
       [  0,   0,  14],
       ...,
       [ 84,  84, 161],
       [ 84,  84, 168],
       [ 84,  84, 175]])

In [3]:
# generate Euler angles each 7 degrees and store in the database
database = SimpleNamespace(euler=f.explore_Euler_space(step=7, lower_bounds=(0, 0, 0), upper_bounds=(89, 89, 179)))
database.euler.shape

(4394, 3)

In [4]:
# Transmission values for lambda 1987.29
Ta = 0.009882
Tb = 0.995815
Tc = 0.596951

# Generate a mesh of values defining the reference transmissión envelope
polar, azimuths = f.regular_S2_grid(n_squared=500)
T = f.Tvalues(trans=(Ta, Tb, Tc), azimuth=azimuths, polar=polar)
x, y, z = f.sph2cart(T, azimuths, polar)

# Generate 16 measures at different azimuth angles
angles = np.arange(0, 360, 45/2)

# initialize variables
T_vals = []
azi_vals = []

for euler in database.euler:
    # rotate
    x2, y2, z2 = f.rotate(coordinates=(x, y, z), euler_ang=euler)

    # extract XY intersection
    xy_vectors = f.extract_XY_section_fast2(x2, y2, z2)

    # get the indexes of specific angles
    indexes = f.find_nearest(xy_vectors['angles'], angles)

    # append values
    T_vals.append(xy_vectors.loc[indexes, ['T']].T.values.tolist()[0])
    azi_vals.append(xy_vectors.loc[indexes, ['angles']].T.values.tolist()[0])

# store values in database
database.T_values = np.array(T_vals)
database.azimuths = np.array(azi_vals)


# GENERATE NOISY (GAUSSIAN NOISE) DATA
# create a random generator
rgn = np.random.default_rng()

# set standard deviation
std_T = 0.03
std_azimuths = 0.5

database.T_noise = database.T_values + rgn.normal(0, std_T, database.T_values.shape)
database.azi_noise = database.azimuths + rgn.normal(0, std_azimuths, database.azimuths.shape)

In [5]:
# check
print(database.euler.shape)
print(database.T_values.shape)
print(database.azimuths.shape)
print(database.T_noise.shape)
print(database.azi_noise.shape)

(4394, 3)
(4394, 16)
(4394, 16)
(4394, 16)
(4394, 16)


In [6]:
# check
print('Euler angles: ', database.euler[-1])
print('T values: ', np.around(database.T_values[-1], 2))
print('Azimuths: ', np.around(database.azimuths[-1], 1))
print('T noise: ', np.around(database.T_noise[-1], 2))
print('Azi_noise: ', np.around(database.azi_noise[-1], 1))

Euler angles:  [ 84  84 175]
T values:  [0.98 0.89 0.74 0.62 0.6  0.68 0.83 0.96 0.98 0.89 0.74 0.62 0.6  0.68
 0.83 0.96]
Azimuths:  [  0.2  22.5  45.2  67.4  90.  112.3 135.  157.5 180.  202.5 225.2 247.5
 270.  292.3 315.  337.5]
T noise:  [0.96 0.87 0.72 0.59 0.64 0.64 0.84 1.01 0.95 0.89 0.77 0.54 0.67 0.73
 0.82 0.96]
Azi_noise:  [  0.4  22.3  45.2  66.9  89.9 111.9 134.6 157.7 179.4 203.1 226.1 247.9
 270.5 292.5 316.  337.3]


## Estimate

In [7]:
# initialize variables 
no_noise = np.empty(database.euler.shape[0])
noise = np.empty(database.euler.shape[0])

for index, orientation in enumerate(database.euler):
    # print('Real:', np.around(orientation, 0))

    measures = np.column_stack((database.T_values[index],
                                database.azimuths[index],
                                np.full_like(database.azimuths[index], 90)))

    estimate_1 = f.find_orientation(measurements=measures, params=(Ta, Tb, Tc))
    no_noise[index] = f.calc_disorientation(orientation, estimate_1.x)

    measures = np.column_stack((database.T_noise[index],
                                database.azi_noise[index],
                                np.full_like(database.azimuths[index], 90)))
    
    estimate_2 = f.find_orientation(measurements=measures, params=(Ta, Tb, Tc))
    noise[index] = f.calc_disorientation(orientation, estimate_2.x)


## Generate dataframe and save data

In [8]:
dataset = pd.DataFrame(
    {
        'Euler1': database.euler[:, 0],
        'Euler2': database.euler[:, 1],
        'Euler3': database.euler[:, 2],
        'misorientation': no_noise,
        'misor_noise': noise
    }
)

dataset

,Euler1,Euler2,Euler3,misorientation,misor_noise
0,0,0,0,0.283,14.323
1,0,0,7,0.178,12.760
2,0,0,14,0.000,0.403
3,0,0,21,0.270,14.540
4,0,0,28,0.272,7.221
...,...,...,...,...,...
4389,84,84,147,0.011,9.681
4390,84,84,154,0.013,7.017
4391,84,84,161,0.009,8.840
4392,84,84,168,0.011,8.646


In [9]:
from datetime import date    
today = date.today().isoformat()

In [10]:
dataset.to_csv(today + '_step7_lamb1987_noise.csv', index=False)

In [11]:
# quick statistical overview
dataset[['misorientation', 'misor_noise']].describe()

,misorientation,misor_noise
count,4394.000000,4394.000000
mean,0.034053,5.242438
std,0.174068,3.545016
min,0.000000,0.061000
25%,0.003000,2.655000
50%,0.004000,4.405500
75%,0.011000,7.003750
max,6.753000,28.140000
